In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e2/sample_submission.csv
/kaggle/input/playground-series-s5e2/train.csv
/kaggle/input/playground-series-s5e2/test.csv
/kaggle/input/playground-series-s5e2/training_extra.csv


In [2]:
import numpy as np
import pandas as pd
import warnings
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pandas as pd
# Suppress Warnings
warnings.filterwarnings("ignore")

In [3]:
train1  = pd.read_csv("/kaggle/input/playground-series-s5e2/train.csv") 
train2 = pd.read_csv("/kaggle/input/playground-series-s5e2/training_extra.csv") 
test_csv = pd.read_csv('/kaggle/input/playground-series-s5e2/test.csv')
train = pd.concat([train1, train2], ignore_index=True)

train.head()


,id,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg),Price
0,0,Jansport,Leather,Medium,7.0,Yes,No,Tote,Black,11.611723,112.15875
1,1,Jansport,Canvas,Small,10.0,Yes,Yes,Messenger,Green,27.078537,68.88056
2,2,Under Armour,Leather,Small,2.0,Yes,No,Messenger,Red,16.643760,39.17320
3,3,Nike,Nylon,Small,8.0,Yes,No,Messenger,Green,12.937220,80.60793
4,4,Adidas,Canvas,Medium,1.0,Yes,Yes,Messenger,Green,17.749338,86.02312


In [4]:
print(len(train))
print(len(test_csv))

3994318
200000


In [5]:
# train = train.drop(columns=["Color","Brand",'Material'])  # Features
# test = test_csv.drop(columns=["Color","Brand",'Material'])  

In [6]:
test = test_csv

In [7]:
for col in train.columns:
    print(f"{col}: {train[col].nunique()} unique values")

id: 3994318 unique values
Brand: 5 unique values
Material: 4 unique values
Size: 3 unique values
Compartments: 10 unique values
Laptop Compartment: 2 unique values
Waterproof: 2 unique values
Style: 3 unique values
Color: 6 unique values
Weight Capacity (kg): 1920345 unique values
Price: 48358 unique values


In [8]:
print("Number of Duplicated rows: ",train.duplicated().sum())

print("Null values count for each column: ",train.isnull().sum())
train=train.dropna()
print("Null values count for each column: ",train.isnull().sum())


Number of Duplicated rows:  0
Null values count for each column:  id                           0
Brand                   126758
Material                110962
Size                     87785
Compartments                 0
Laptop Compartment       98533
Waterproof               94324
Style                   104180
Color                   133617
Weight Capacity (kg)      1808
Price                        0
dtype: int64
Null values count for each column:  id                      0
Brand                   0
Material                0
Size                    0
Compartments            0
Laptop Compartment      0
Waterproof              0
Style                   0
Color                   0
Weight Capacity (kg)    0
Price                   0
dtype: int64


In [9]:

# Fill numerical columns with the median
test['Weight Capacity (kg)'].fillna(test['Weight Capacity (kg)'].median(), inplace=True)

# Fill categorical columns with the most frequent value (mode)
categorical_cols = ['Size', 'Laptop Compartment', 'Waterproof', 'Style']
for col in categorical_cols:
    test[col].fillna(test[col].mode()[0], inplace=True)  # Fill with most frequent value

print("Null values count for each column: ",test.isnull().sum())


Null values count for each column:  id                         0
Brand                   6227
Material                5613
Size                       0
Compartments               0
Laptop Compartment         0
Waterproof                 0
Style                      0
Color                   6785
Weight Capacity (kg)       0
dtype: int64


In [10]:
label_encoders = {}
for col in ["Size", "Compartments", "Laptop Compartment", "Waterproof"]:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])
    test[col]=le.transform(test[col])
    label_encoders[col] = le

# Apply One-Hot Encoding to categorical features
train = pd.get_dummies(train, columns=[ "Style","Color","Brand",'Material'])
test = pd.get_dummies(test, columns=[ "Style","Color","Brand",'Material'])

train.head()

,id,Size,Compartments,Laptop Compartment,Waterproof,Weight Capacity (kg),Price,Style_Backpack,Style_Messenger,Style_Tote,...,Color_Red,Brand_Adidas,Brand_Jansport,Brand_Nike,Brand_Puma,Brand_Under Armour,Material_Canvas,Material_Leather,Material_Nylon,Material_Polyester
0,0,1,6,1,0,11.611723,112.15875,False,False,True,...,False,False,True,False,False,False,False,True,False,False
1,1,2,9,1,1,27.078537,68.88056,False,True,False,...,False,False,True,False,False,False,True,False,False,False
2,2,2,1,1,0,16.643760,39.17320,False,True,False,...,True,False,False,False,False,True,False,True,False,False
3,3,2,7,1,0,12.937220,80.60793,False,True,False,...,False,False,False,True,False,False,False,False,True,False
4,4,1,0,1,1,17.749338,86.02312,False,True,False,...,False,True,False,False,False,False,True,False,False,False


In [11]:
test.head()

,id,Size,Compartments,Laptop Compartment,Waterproof,Weight Capacity (kg),Style_Backpack,Style_Messenger,Style_Tote,Color_Black,...,Color_Red,Brand_Adidas,Brand_Jansport,Brand_Nike,Brand_Puma,Brand_Under Armour,Material_Canvas,Material_Leather,Material_Nylon,Material_Polyester
0,300000,2,1,0,0,20.671147,False,False,True,False,...,False,False,False,False,True,False,False,True,False,False
1,300001,1,6,0,1,13.564105,True,False,False,False,...,False,False,False,True,False,False,True,False,False,False
2,300002,0,8,0,1,11.809799,False,True,False,False,...,False,True,False,False,False,False,True,False,False,False
3,300003,0,0,1,0,18.477036,False,True,False,False,...,False,True,False,False,False,False,False,False,True,False
4,300004,0,1,1,1,9.907953,False,False,True,True,...,False,False,False,False,False,False,False,False,True,False


In [12]:
# Define the target column
target_col = "Price"  # Replace with your actual target column name

# Separate features (X) and target (y)
X_train = train.drop(columns=[target_col,'id'])
y_train = train[target_col]

# Save the Id 
idtest = test['id']

# drop the id column from the data
X_test = test.drop(columns=['id'])

# Print shape to verify
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)


X_train shape: (3288503, 23)
y_train shape: (3288503,)
X_test shape: (200000, 23)


In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


# Separate Features & Target
# X = train.drop(columns=["Price","id"])  # Features
# y = train["Price"]  # Target Variable

# # **Step 1: Split Data**
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# **Step 2: Standardize Only Numerical Features**
scaler = StandardScaler()
num_features = ["Weight Capacity (kg)"]  # Only numerical features

# Fit on train data only
X_train[num_features] = scaler.fit_transform(X_train[num_features])

# Transform test data using the same scaler
X_test[num_features] = scaler.transform(X_test[num_features])

X_train.head()

,Size,Compartments,Laptop Compartment,Waterproof,Weight Capacity (kg),Style_Backpack,Style_Messenger,Style_Tote,Color_Black,Color_Blue,...,Color_Red,Brand_Adidas,Brand_Jansport,Brand_Nike,Brand_Puma,Brand_Under Armour,Material_Canvas,Material_Leather,Material_Nylon,Material_Polyester
0,1,6,1,0,-0.916410,False,False,True,True,False,...,False,False,True,False,False,False,False,True,False,False
1,2,9,1,1,1.303308,False,True,False,False,False,...,False,False,True,False,False,False,True,False,False,False
2,2,1,1,0,-0.194238,False,True,False,False,False,...,True,False,False,False,False,True,False,True,False,False
3,2,7,1,0,-0.726181,False,True,False,False,False,...,False,False,False,True,False,False,False,False,True,False
4,1,0,1,1,-0.035571,False,True,False,False,False,...,False,True,False,False,False,False,True,False,False,False


In [14]:

X_test.head()

,Size,Compartments,Laptop Compartment,Waterproof,Weight Capacity (kg),Style_Backpack,Style_Messenger,Style_Tote,Color_Black,Color_Blue,...,Color_Red,Brand_Adidas,Brand_Jansport,Brand_Nike,Brand_Puma,Brand_Under Armour,Material_Canvas,Material_Leather,Material_Nylon,Material_Polyester
0,2,1,0,0,0.383752,False,False,True,False,False,...,False,False,False,False,True,False,False,True,False,False
1,1,6,0,1,-0.636214,True,False,False,False,False,...,False,False,False,True,False,False,True,False,False,False
2,0,8,0,1,-0.887983,False,True,False,False,True,...,False,True,False,False,False,False,True,False,False,False
3,0,0,1,0,0.068864,False,True,False,False,False,...,False,True,False,False,False,False,False,False,True,False
4,0,1,1,1,-1.160926,False,False,True,True,False,...,False,False,False,False,False,False,False,False,True,False


In [15]:
# import numpy as np
# import xgboost as xgb
# from sklearn.datasets import load_breast_cancer
# from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
# from sklearn.metrics import accuracy_score


# # Initialize XGBoost regressor
# xgb_reg = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

# # Define a smaller parameter grid
# param_dist = {
#     'n_estimators': [100, 200],
#     'learning_rate': [0.05, 0.1],
#     'max_depth': [3, 5],
#     'subsample': [0.8, 1.0]
# }


In [16]:

# from sklearn.metrics import mean_squared_error

# # Randomized Search with fewer fits
# random_search = RandomizedSearchCV(
#     estimator=xgb_reg,
#     param_distributions=param_dist,
#     n_iter=10,   # Limits the number of total fits
#     cv=3,        # 3-fold cross-validation
#     scoring='neg_mean_squared_error',  # Use MSE for regression
#     n_jobs=-1,
#     verbose=1,
#     random_state=42
# )

# # Fit the model
# random_search.fit(X_train, y_train)

# # Best parameters and test evaluation
# print("Best Parameters:", random_search.best_params_)

# # Evaluate on test set
# best_xgb = random_search.best_estimator_
# y_pred = best_xgb.predict(X_test)
# mse = mean_squared_error(y_test, y_pred)
# print("Test MSE:", mse)
# print("Test RMSE:", np.sqrt(mse))  # Root Mean Squared Error

In [17]:
import xgboost as xgb  # ✅ Import XGBoost

# Train the final model with the best parameters
best_xgb = xgb.XGBRegressor(
    subsample=1.0, 
    n_estimators=100, 
    max_depth=3, 
    learning_rate=0.05,
    objective='reg:squarederror',
    random_state=42
)

best_xgb.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [18]:
from sklearn.metrics import r2_score

# Predict on test set
y_pred = best_xgb.predict(X_test)

In [19]:
y_pred[:10]

array([81.768524, 82.998024, 81.45314 , 81.50386 , 78.9423  , 81.48979 ,
       82.030975, 83.32164 , 82.410515, 79.78388 ], dtype=float32)

In [20]:
import joblib

# Save the model
joblib.dump(best_xgb, 'xgboost_model.pkl')

# Load the model
loaded_model = joblib.load('xgboost_model.pkl')




In [21]:
# Prepare submission file
submission = pd.DataFrame({'id': idtest, 'Price': y_pred})
submission.to_csv("submission.csv", index=False)

In [22]:
submission.head()

,id,Price
0,300000,81.768524
1,300001,82.998024
2,300002,81.453140
3,300003,81.503860
4,300004,78.942299
